In [0]:
#https://www.kaggle.com/mksaad/arabic-sentiment-analysis-in-tweets-nb-bow
#https://github.com/saobou/arabic-text-preprocessing/blob/master/Preprocess.ipynb
import numpy as np 
import pandas as pd

In [0]:
import nltk
from nltk import NaiveBayesClassifier
from nltk.metrics.scores import f_measure, precision, recall
import collections

In [0]:
pos_dataset = pd.read_csv('train_Arabic_tweets_positive_20190413.tsv',sep='\t', encoding='utf-8')

In [0]:
neg_dataset = pd.read_csv('train_Arabic_tweets_negative_20190413.tsv',sep='\t', encoding="utf-8")

In [5]:
neg_dataset.head

<bound method NDFrame.head of       Sentiment                                               Text
0           neg  اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم...
1           neg  توقعت اذا جات داريا بشوفهم كاملين بس لي للحين ...
2           neg  #الاهلي_الهلال اكتب توقعك لنتيجة لقاء الهلال و...
3           neg  نعمة المضادات الحيوية . تضع قطرة💧مضاد بنسلين ع...
4           neg                             الدودو جايه تكمل علي 💔
...         ...                                                ...
22509       neg  كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...
22510       neg                                  احسدك على الايم 💔
22511       neg                            لأول مرة ما بنكون سوا 💔
22512       neg                                 بقله ليش يا واطي 🤔
22513       neg  قد طال صبري في النوى إذ تركتني كئيبا ؛ غريبا ب...

[22514 rows x 2 columns]>

In [6]:
pos_dataset['Text'][1]

'وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال في روحك أماالمنبهرون بالمظا…'

In [0]:
pos = pd.DataFrame(columns=['Text','Label'])

In [0]:
neg = pd.DataFrame(columns=['Text','Label'])

In [0]:
pos_tweets=[]
neg_tweets=[]
pos_sent=[]
neg_sent=[]
for i in range(len(pos_dataset)):
  pos_tweets.append(pos_dataset['Text'][i])
  pos_sent.append(1)

for i in range(len(neg_dataset)):
  neg_tweets.append(neg_dataset['Text'][i])
  neg_sent.append(0)

In [10]:
pos_tweets[1]

'وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال في روحك أماالمنبهرون بالمظا…'

In [11]:
neg_tweets[1]

'توقعت اذا جات داريا بشوفهم كاملين بس لي للحين احس فيه احد ناقصهم 💔 #Avlu'

In [0]:
text=[]
Label=[]
for tweet in pos_tweets:
  text.append(tweet)
  Label.append(1)
for tweet in neg_tweets:
  text.append(tweet)
  Label.append(0)

In [0]:
data = pd.DataFrame(columns=['Text','Labels'])

In [0]:
data['Text'] = text
data['Labels'] = Label

In [16]:
data

,Text,Labels
0,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,1
1,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,1
2,من الخير نفسه 💛,1
3,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,1
4,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,1
...,...,...
45270,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...,0
45271,احسدك على الايم 💔,0
45272,لأول مرة ما بنكون سوا 💔,0
45273,بقله ليش يا واطي 🤔,0


### arabic text cleaning

In [23]:
!pip install Tashaphyne
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from textblob import TextBlob
import re

from tashaphyne.stemming import ArabicLightStemmer
from nltk.stem.isri import ISRIStemmer

stops = set(stopwords.words("arabic"))
stop_word_comp = {"،","آض","آمينَ","آه","آهاً","آي","أ","أب","أجل","أجمع","أخ","أخذ","أصبح","أضحى","أقبل","أقل","أكثر","ألا","أم","أما","أمامك","أمامكَ","أمسى","أمّا","أن","أنا","أنت","أنتم","أنتما","أنتن","أنتِ","أنشأ","أنّى","أو","أوشك","أولئك","أولئكم","أولاء","أولالك","أوّهْ","أي","أيا","أين","أينما","أيّ","أَنَّ","أََيُّ","أُفٍّ","إذ","إذا","إذاً","إذما","إذن","إلى","إليكم","إليكما","إليكنّ","إليكَ","إلَيْكَ","إلّا","إمّا","إن","إنّما","إي","إياك","إياكم","إياكما","إياكن","إيانا","إياه","إياها","إياهم","إياهما","إياهن","إياي","إيهٍ","إِنَّ","ا","ابتدأ","اثر","اجل","احد","اخرى","اخلولق","اذا","اربعة","ارتدّ","استحال","اطار","اعادة","اعلنت","اف","اكثر","اكد","الألاء","الألى","الا","الاخيرة","الان","الاول","الاولى","التى","التي","الثاني","الثانية","الذاتي","الذى","الذي","الذين","السابق","الف","اللائي","اللاتي","اللتان","اللتيا","اللتين","اللذان","اللذين","اللواتي","الماضي","المقبل","الوقت","الى","اليوم","اما","امام","امس","ان","انبرى","انقلب","انه","انها","او","اول","اي","ايار","ايام","ايضا","ب","بات","باسم","بان","بخٍ","برس","بسبب","بسّ","بشكل","بضع","بطآن","بعد","بعض","بك","بكم","بكما","بكن","بل","بلى","بما","بماذا","بمن","بن","بنا","به","بها","بي","بيد","بين","بَسْ","بَلْهَ","بِئْسَ","تانِ","تانِك","تبدّل","تجاه","تحوّل","تلقاء","تلك","تلكم","تلكما","تم","تينك","تَيْنِ","تِه","تِي","ثلاثة","ثم","ثمّ","ثمّة","ثُمَّ","جعل","جلل","جميع","جير","حار","حاشا","حاليا","حاي","حتى","حرى","حسب","حم","حوالى","حول","حيث","حيثما","حين","حيَّ","حَبَّذَا","حَتَّى","حَذارِ","خلا","خلال","دون","دونك","ذا","ذات","ذاك","ذانك","ذانِ","ذلك","ذلكم","ذلكما","ذلكن","ذو","ذوا","ذواتا","ذواتي","ذيت","ذينك","ذَيْنِ","ذِه","ذِي","راح","رجع","رويدك","ريث","رُبَّ","زيارة","سبحان","سرعان","سنة","سنوات","سوف","سوى","سَاءَ","سَاءَمَا","شبه","شخصا","شرع","شَتَّانَ","صار","صباح","صفر","صهٍ","صهْ","ضد","ضمن","طاق","طالما","طفق","طَق","ظلّ","عاد","عام","عاما","عامة","عدا","عدة","عدد","عدم","عسى","عشر","عشرة","علق","على","عليك","عليه","عليها","علًّ","عن","عند","عندما","عوض","عين","عَدَسْ","عَمَّا","غدا","غير","ـ","ف","فان","فلان","فو","فى","في","فيم","فيما","فيه","فيها","قال","قام","قبل","قد","قطّ","قلما","قوة","كأنّما","كأين","كأيّ","كأيّن","كاد","كان","كانت","كذا","كذلك","كرب","كل","كلا","كلاهما","كلتا","كلم","كليكما","كليهما","كلّما","كلَّا","كم","كما","كي","كيت","كيف","كيفما","كَأَنَّ","كِخ","لئن","لا","لات","لاسيما","لدن","لدى","لعمر","لقاء","لك","لكم","لكما","لكن","لكنَّما","لكي","لكيلا","للامم","لم","لما","لمّا","لن","لنا","له","لها","لو","لوكالة","لولا","لوما","لي","لَسْتَ","لَسْتُ","لَسْتُم","لَسْتُمَا","لَسْتُنَّ","لَسْتِ","لَسْنَ","لَعَلَّ","لَكِنَّ","لَيْتَ","لَيْسَ","لَيْسَا","لَيْسَتَا","لَيْسَتْ","لَيْسُوا","لَِسْنَا","ما","ماانفك","مابرح","مادام","ماذا","مازال","مافتئ","مايو","متى","مثل","مذ","مساء","مع","معاذ","مقابل","مكانكم","مكانكما","مكانكنّ","مكانَك","مليار","مليون","مما","ممن","من","منذ","منها","مه","مهما","مَنْ","مِن","نحن","نحو","نعم","نفس","نفسه","نهاية","نَخْ","نِعِمّا","نِعْمَ","ها","هاؤم","هاكَ","هاهنا","هبّ","هذا","هذه","هكذا","هل","هلمَّ","هلّا","هم","هما","هن","هنا","هناك","هنالك","هو","هي","هيا","هيت","هيّا","هَؤلاء","هَاتانِ","هَاتَيْنِ","هَاتِه","هَاتِي","هَجْ","هَذا","هَذانِ","هَذَيْنِ","هَذِه","هَذِي","هَيْهَاتَ","و","و6","وا","واحد","واضاف","واضافت","واكد","وان","واهاً","واوضح","وراءَك","وفي","وقال","وقالت","وقد","وقف","وكان","وكانت","ولا","ولم","ومن","مَن","وهو","وهي","ويكأنّ","وَيْ","وُشْكَانََ","يكون","يمكن","يوم","ّأيّان"}
ArListem = ArabicLightStemmer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
def to_arabic(text):
    return ds.transliterate(text)

def stem(text):
    zen = TextBlob(text)
    words = zen.words
    cleaned = list()
    for w in words:
        ArListem.light_stem(w)
        cleaned.append(ArListem.get_root())
    return " ".join(cleaned)

In [0]:
import pyarabic.araby as araby
def normalizeArabic(text):
    text = text.strip()
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    text = re.sub(r'(.)\1+', r"\1\1", text) # Remove longation
    return araby.strip_tashkeel(text)

In [0]:
def remove_stop_words(text):
    zen = TextBlob(text)
    words = zen.words
    return " ".join([w for w in words if not w in stops and not w in stop_word_comp and len(w) >= 2])

### Dealing with Hashtags in string

In [0]:
def split_hashtag_to_words(tag):
    tag = tag.replace('#','')
    tags = tag.split('_')
    if len(tags) > 1 :
        
        return tags
    pattern = re.compile(r"[A-Z][a-z]+|\d+|[A-Z]+(?![a-z])")
    return pattern.findall(tag)

def clean_hashtag(text):
    words = text.split()
    text = list()
    for word in words:
        if is_hashtag(word):
            text.extend(extract_hashtag(word))
        else:
            text.append(word)
    return " ".join(text)
def is_hashtag(word):
    if word.startswith("#"):
        return True
    else:
        return False
def extract_hashtag(text):
    
    hash_list = ([re.sub(r"(\W+)$", "", i) for i in text.split() if i.startswith("#")])
    word_list = []
    for word in hash_list :
        word_list.extend(split_hashtag_to_words(word))
    return word_list

### MISC

In [30]:
import re
import pyarabic.araby as araby

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# test sample
tweet1 = u'جهود تُبذل عامًا بعد عام تعبّر عن ترابط الجسد الواحد لخدمة ضيوف الرحمن؛ تطبيق ترجمان الفائز في #هاكاثون_الحج العام الماضي، ينطلق اليوم ليشارك الحجاج أيامهم الروحانية ويساعدهم في ترجمة اللوح الإرشادية في منطقة الحرم دون الحاجة إلى انترنت @reem. #حج_ذكي'



# removes usernames inside a tweet text caused by to mentions or reply tweets
def remove_usernames(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)

    return input_txt

# removes arabic letters tashkeel like (ِ ًٍ ٌ  ّ ْ )
def remove_tashkeel(input_txt):
    return araby.strip_tashkeel(input_txt)

# removes a decorative letter (ـ) which has no affect on words meaning
def remove_tatweel(input_txt):
    return araby.strip_tatweel(input_txt)

# normalizes the different forms of the letter hamza (ئ ؤ) into a single form (ء)
def normalize_hamza(input_txt):
    r = re.findall(u'ئ',input_txt);
    e = re.findall(u'ؤ',input_txt);
    for i in r:
        input_txt= re.sub(i, u'ء', input_txt)
    for i in e:
        input_txt= re.sub(i, u'ء', input_txt)
    return input_txt

# normalizes the different forms of the letter alef (آ أ إ) into a single form (ا)
def normalize_alef(input_txt):
    r = re.findall(u'أ',input_txt);
    e = re.findall(u'إ',input_txt);
    o = re.findall(u'آ', input_txt);
    for i in r:
        input_txt= re.sub(i, u'ا', input_txt)
    for i in e:
        input_txt= re.sub(i, u'ا', input_txt)
    for i in o:
        input_txt= re.sub(i, u'ا', input_txt)
    return input_txt

# normalizes the different forms of the letter yeh (ي ى) into a single form (ى)
def normalize_yeh(input_txt):
    r = re.findall(u'ي', input_txt)
    for i in r:
        input_txt = re.sub(i, u'ى', input_txt)
    return input_txt

# normalizes the different forms of the letter heh (ه ة) into a single form (ة)
def normalize_heh(input_txt):
    r = re.findall(u'ه', input_txt)
    for i in r:
        input_txt = re.sub(i, u'ة', input_txt)
    return input_txt

# combining all preprocessing functions together for testing samples
def full_preprocessing_steps(tweet):

    print("Tweet before preprocessing: " + tweet)

    usernames_free = remove_usernames(tweet,  "@[\w]*")
    tashkeel_free = remove_tashkeel(usernames_free)
    tatweel_free = remove_tatweel(tashkeel_free)
    hamza_normalized = normalize_hamza(tatweel_free)
    alef_normalized = normalize_alef(hamza_normalized)
    yeh_normalized = normalize_yeh(alef_normalized)
    heh_normalized = normalize_heh(yeh_normalized)

    print("Tweet after preprocessing: " + heh_normalized)

full_preprocessing_steps(tweet1)

Tweet before preprocessing: جهود تُبذل عامًا بعد عام تعبّر عن ترابط الجسد الواحد لخدمة ضيوف الرحمن؛ تطبيق ترجمان الفائز في #هاكاثون_الحج العام الماضي، ينطلق اليوم ليشارك الحجاج أيامهم الروحانية ويساعدهم في ترجمة اللوح الإرشادية في منطقة الحرم دون الحاجة إلى انترنت @reem. #حج_ذكي
Tweet after preprocessing: جةود تبذل عاما بعد عام تعبر عن ترابط الجسد الواحد لخدمة ضىوف الرحمن؛ تطبىق ترجمان الفاءز فى #ةاكاثون_الحج العام الماضى، ىنطلق الىوم لىشارك الحجاج اىامةم الروحانىة وىساعدةم فى ترجمة اللوح الارشادىة فى منطقة الحرم دون الحاجة الى انترنت . #حج_ذكى
